In [1]:
# prepare needed environment on Colab
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://apache.mirror.digionline.de/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
!tar xf spark-3.1.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

import findspark
findspark.init()

In [3]:
!pip install elephas

     |████████████████████████████████| 337kB 24.7MB/s 
     |████████████████████████████████| 2.9MB 47.6MB/s 
     |████████████████████████████████| 212.4MB 89kB/s 
     |████████████████████████████████| 51kB 4.9MB/s 
     |████████████████████████████████| 204kB 57.5MB/s 
  Created wheel for elephas: filename=elephas-2.1.0-cp37-none-any.whl size=27614 sha256=357be410e00c8638a7f3d544eb7252b8d9f2a7c2f68bd0ec9e186eb55d1fd2d4
  Stored in directory: /root/.cache/pip/wheels/b4/8e/f7/afeaa15a424e0df01ff445dea2ac4cfddda282329494bbb027
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=fc7078f945c0ccb7481b4081e9b0879ffe51758b0b78b9e6e4b202ac05d24ed4
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built elephas pyspark
ERROR: tensorflow 2.5.0 has requirement h5py~=3.1.0, but you'll have h5py 2.10.0 which is incompatible.
  Found existing installation: h5py 3.1.0
    Uninstalli

In [4]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from elephas.spark_model import SparkModel
from elephas.utils.rdd_utils import to_simple_rdd

from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical

# Define basic parameters
batch_size = 64
nb_classes = 10
epochs = 1

# Create Spark context
conf = SparkConf().setAppName('Mnist_Spark_MLP').setMaster('local[*]')
sc = SparkContext(conf=conf)

# Load data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train /= 255
x_test /= 255

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices
y_train = to_categorical(y_train, nb_classes)
y_test = to_categorical(y_test, nb_classes)

model = Sequential()
model.add(Dense(128, input_dim=784))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(10))
model.add(Activation('softmax'))

sgd = SGD(lr=0.1)
model.compile(sgd, 'categorical_crossentropy', ['acc'])

# Build RDD from numpy features and labels
rdd = to_simple_rdd(sc, x_train, y_train)

# Initialize SparkModel from tensorflow.keras model and Spark context
spark_model = SparkModel(model, frequency='epoch', mode='asynchronous')

# Train Spark model
spark_model.fit(rdd, epochs=epochs, batch_size=batch_size, verbose=0, validation_split=0.1)

# Evaluate Spark model by evaluating the underlying model
score = spark_model.evaluate(x_test, y_test, verbose=2)
print('Test accuracy:', score[1])

11493376/11490434 [==============================] - 0s 0us/step
60000 train samples
10000 test samples


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


>>> Fit model
 * Serving Flask app "elephas.parameter.server" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://172.28.0.2:4000/ (Press CTRL+C to quit)


>>> Initialize workers
>>> Distribute load


172.28.0.2 - - [31/May/2021 18:16:24] "GET /parameters HTTP/1.1" 200 -
172.28.0.2 - - [31/May/2021 18:16:24] "GET /parameters HTTP/1.1" 200 -
172.28.0.2 - - [31/May/2021 18:16:26] "POST /update HTTP/1.1" 200 -
172.28.0.2 - - [31/May/2021 18:16:26] "POST /update HTTP/1.1" 200 -
172.28.0.2 - - [31/May/2021 18:16:27] "GET /parameters HTTP/1.1" 200 -


>>> Async training complete.
Test accuracy: 0.9187291860580444


In [5]:
'''
Possible solution: https://stackoverflow.com/questions/61326144/pyspark-pipeline-fitdf-method-give-picklingerror-could-not-serialize-object  
!pip install q keras==2.2.4
!pip install q tensorflow==1.14.0
---------------------------------------------------------------------------
ValueError: Cell is empty


ValueError                                Traceback (most recent call last)

/content/spark-3.1.1-bin-hadoop2.7/python/pyspark/serializers.py in dumps(self, obj)
    436         try:
--> 437             return cloudpickle.dumps(obj, pickle_protocol)
    438         except pickle.PickleError:
---------------------------------------------------------------------------


from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers

from pyspark import SparkContext, SparkConf
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession

from elephas.ml_model import ElephasEstimator
from elephas.ml.adapter import to_data_frame
from elephas.utils.rdd_utils import to_simple_rdd


# Define basic parameters
batch_size = 64
nb_classes = 10
epochs = 1

# Load data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices
y_train = to_categorical(y_train, nb_classes)
y_test = to_categorical(y_test, nb_classes)

spark = SparkSession\
        .builder\
        .appName("keras-elphas-mnist")\
        .getOrCreate()

model = Sequential()
model.add(Dense(128, input_dim=784))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(10))
model.add(Activation('softmax'))

sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(sgd, 'categorical_crossentropy', ['acc'])

# Create Spark context
#conf = SparkConf().setAppName('Mnist_Spark_MLP') # .setMaster('local[8]')
#sc = SparkContext(conf=conf)
sc = spark.sparkContext

# Build RDD from numpy features and labels
df = to_data_frame(sc, x_train, y_train, categorical=True)
test_df = to_data_frame(sc, x_test, y_test, categorical=True)

sgd_conf = optimizers.serialize(sgd)

# Initialize Spark ML Estimator
estimator = ElephasEstimator()
estimator.set_keras_model_config(model.to_yaml())
estimator.set_optimizer_config(sgd_conf)
estimator.set_mode("synchronous")
estimator.set_loss("categorical_crossentropy")
estimator.set_metrics(['acc'])
estimator.set_epochs(epochs)
estimator.set_batch_size(batch_size)
estimator.set_validation_split(0.1)
estimator.set_categorical_labels(True)
estimator.set_nb_classes(nb_classes)

# Fitting a model returns a Transformer
pipeline = Pipeline(stages=[estimator])
fitted_pipeline = pipeline.fit(df)

# Evaluate Spark model by evaluating the underlying model
prediction = fitted_pipeline.transform(test_df)
pnl = prediction.select("label", "prediction")
pnl.show(100)

prediction_and_label = pnl.rdd.map(lambda row: (row.label, row.prediction))
metrics = MulticlassMetrics(prediction_and_label)
print(metrics.precision())
print(metrics.recall())
spark.stop()
'''

'\nPossible solution: https://stackoverflow.com/questions/61326144/pyspark-pipeline-fitdf-method-give-picklingerror-could-not-serialize-object  \n!pip install q keras==2.2.4\n!pip install q tensorflow==1.14.0\n---------------------------------------------------------------------------\nValueError: Cell is empty\n\n\nValueError                                Traceback (most recent call last)\n\n/content/spark-3.1.1-bin-hadoop2.7/python/pyspark/serializers.py in dumps(self, obj)\n    436         try:\n--> 437             return cloudpickle.dumps(obj, pickle_protocol)\n    438         except pickle.PickleError:\n---------------------------------------------------------------------------\n\n\nfrom tensorflow.keras.datasets import mnist\nfrom tensorflow.keras.models import Sequential\nfrom tensorflow.keras.layers import Dense, Dropout, Activation\nfrom tensorflow.keras.utils import to_categorical\nfrom tensorflow.keras import optimizers\n\nfrom pyspark import SparkContext, SparkConf\nfrom p